In [ ]:
config = Config()

# Define the audio input (1D) modality
config.architecture = './model_components'  
config.model_type = 'inr_modules.CombinedINR'

config.model_config = Config()
config.model_config.in_size = 1  # Since audio is 1D
config.model_config.out_size = 1  # For output signals

config.model_config.terms = [
    ('inr_modules.MLPINR.from_config',{
        'hidden_size': 256,
        'num_layers': 5,
        'layer_type': 'inr_layers.FinerLayer',  # Use your FINER model here
        'num_splits': 1,
        'use_complex': False,
        'activation_kwargs': {'w0': 30.0}  # You can adjust this
    })
]

# Training setup for audio modality
config.trainer_module = './inr_utils/' 
config.trainer_type = 'training.train_inr'

# Set up for audio data input
config.target_function = 'audio.AudioSignal'  # Assuming you have an audio data handler
config.target_function_config = {
    'audio_file': './path_to_audio_file.wav',  # Set the path to the audio file you want to use
    'scale_to_01': True,
}

config.loss_function = 'losses.scaled_mse_loss'
config.sampler = ('sampling.GridSubsetSampler',{
    'size': [audio_length],  # Define the length of the audio signal
    'batch_size': 2000,
    'allow_duplicates': False,
})

config.optimizer = 'adam'
config.optimizer_config = {
    'learning_rate': 1.5e-4
}
config.steps = 40000
config.use_wandb = False  # You can enable this if you're logging with Weights & Biases

config.after_step_callback = 'callbacks.ComposedCallback'
config.after_step_callback_config = {
    'callbacks': [
        ('callbacks.print_loss', {'after_every':400}),
        'callbacks.report_loss',
        ('callbacks.MetricCollectingCallback', {'metric_collector':'metrics.MetricCollector'}),
        'callbacks.raise_error_on_nan'
    ],
    'show_logs': False
}

config.metric_collector_config = {
    'metrics': [
        ('metrics.MSEOnFixedGrid', {'grid': [audio_length], 'batch_size': audio_length, 'frequency': 'every_n_batches'})
    ],
    'batch_frequency': 400,
    'epoch_frequency': 1
}
